In [4]:
import numpy as np
from scipy import sparse



In [5]:
data = np.loadtxt('data/u.data')

def load():
    ij = data[:, :2]
    ij -= 1 # original data is in 1-based system
    values = data[:, 2]
    reviews = sparse.csc_matrix((values, ij.T)).astype(float)
    return reviews.toarray()

array([[  1.96000000e+02,   2.42000000e+02,   3.00000000e+00,
          8.81250949e+08],
       [  1.86000000e+02,   3.02000000e+02,   3.00000000e+00,
          8.91717742e+08],
       [  2.20000000e+01,   3.77000000e+02,   1.00000000e+00,
          8.78887116e+08],
       ..., 
       [  2.76000000e+02,   1.09000000e+03,   1.00000000e+00,
          8.74795795e+08],
       [  1.30000000e+01,   2.25000000e+02,   2.00000000e+00,
          8.82399156e+08],
       [  1.20000000e+01,   2.03000000e+02,   3.00000000e+00,
          8.79959583e+08]])

In [9]:
reviews

NameError: name 'reviews' is not defined